In [1]:
from TrendPy.webApp import App


In [2]:
a = App()

a.show()


HTML(value='<h3 align="center">Upload a .csv file here<h3>')

FigureWidget({
    'data': [{'mode': 'markers', 'name': 'datapoints', 'type': 'scatter', 'uid': '22d7a00e-869d…

**Note:** If a manual trend is selected, then a 3 parameters expression is expected: e.g. ```a*x**2+b*x+c```